In [1]:
import json

# Specify the file path
file_path = './data/journey_plan_2024-08-07_11-43.json'

# Read the JSON file
with open(file_path, 'r') as file:
    journey_plan = json.load(file)

journey_plan

{'requestParameters': {'date': '2024-08-07',
  'mode': 'TRANSIT,WALK',
  'showIntermediateStops': 'True',
  'fromPlace': '41.423043,2.184006',
  'toPlace': '41.406232,2.192273',
  'time': '11:43'},
 'plan': {'date': 1723023780000,
  'from': {'name': 'Origin',
   'lon': 2.184006,
   'lat': 41.423043,
   'vertexType': 'NORMAL'},
  'to': {'name': 'Destination',
   'lon': 2.192273,
   'lat': 41.406232,
   'vertexType': 'NORMAL'},
  'itineraries': [{'duration': 2032,
    'startTime': 1723023780000,
    'endTime': 1723025812000,
    'walkTime': 2032,
    'transitTime': 0,
    'waitingTime': 0,
    'walkDistance': 2437.03,
    'walkLimitExceeded': False,
    'generalizedCost': 5111,
    'elevationLost': 56.45,
    'elevationGained': 3.2,
    'transfers': 0,
    'fare': {'fare': {}, 'details': {}},
    'legs': [{'startTime': 1723023780000,
      'endTime': 1723025812000,
      'departureDelay': 0,
      'arrivalDelay': 0,
      'realTime': False,
      'distance': 2437.03,
      'generalizedCo

In [2]:
import json
import pandas as pd
import uuid

def process_journey_plan(json_file):
    """
    Reads a JSON file containing journey plan data, extracts routes, and saves the information to a CSV file.

    Parameters:
        file_path (str): Path to the input JSON file.
        output_csv (str): Path to the output CSV file.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted route information.
    """
    # Extract route information
    routes = []
    if 'plan' in json_file and 'itineraries' in json_file['plan']:
        for itinerary in json_file['plan']['itineraries']:
            route = {
                'id': str(uuid.uuid4()),  # Unique identifier
                'duration': itinerary.get('duration'),
                'transfers': itinerary.get('transfers'),
                'legs': [],
                'modes': set()  # Use a set to collect unique modes
            }

            for leg in itinerary.get('legs', []):
                leg_info = {
                    'mode': leg.get('mode'),
                    'start_time': leg.get('startTime'),
                    'end_time': leg.get('endTime'),
                    'from': leg.get('from', {}).get('name'),
                    'to': leg.get('to', {}).get('name'),
                    'route': leg.get('route'),
                    'distance': leg.get('distance'),
                    'agency': leg.get('agencyName')
                }
                route['legs'].append(leg_info)
                route['modes'].add(leg.get('mode'))  # Add the mode to the set

            route['modes'] = list(route['modes'])  # Convert the set to a list
            routes.append(route)

    # Normalize the routes data into a DataFrame
    routes_df = pd.json_normalize(routes, 'legs', ['id', 'duration', 'transfers', 'modes'])

    # Sort values by id and start_time
    routes_df = routes_df.sort_values(by=['id', 'start_time'])

    return routes_df


# Process the journey plan and output the CSV
routes_df = process_journey_plan(journey_plan)

In [3]:
routes_df

,mode,start_time,end_time,from,to,route,distance,agency,id,duration,transfers,modes
4,WALK,1723024075000,1723024401000,Origin,Meridiana - La Sagrera,,347.73,None,1beafd31-9faf-4b91-ab44-bd771f251999,1467,0,"[WALK, BUS]"
5,BUS,1723024416000,1723024864000,Meridiana - La Sagrera,Meridiana - Aragó,<M> Clot / Ciutat Meridiana,1551.03,TMB,1beafd31-9faf-4b91-ab44-bd771f251999,1467,0,"[WALK, BUS]"
6,WALK,1723024879000,1723025542000,Meridiana - Aragó,Destination,,694.73,None,1beafd31-9faf-4b91-ab44-bd771f251999,1467,0,"[WALK, BUS]"
7,WALK,1723024027000,1723024429000,Origin,La Sagrera,,552.96,None,90c361ee-8e79-491b-a150-fb801f08cd71,1573,0,"[WALK, SUBWAY]"
8,SUBWAY,1723024444000,1723024602000,La Sagrera,Clot,Hospital de Bellvitge - Fondo,1357.41,TMB,90c361ee-8e79-491b-a150-fb801f08cd71,1573,0,"[WALK, SUBWAY]"
9,WALK,1723024617000,1723025600000,Clot,Destination,,1008.75,None,90c361ee-8e79-491b-a150-fb801f08cd71,1573,0,"[WALK, SUBWAY]"
1,WALK,1723024025000,1723024215000,Origin,Pl dels Indians,,140.02,None,bc893ea5-7fd2-454c-9d30-b1dde596a8e2,1512,0,"[WALK, BUS]"
2,BUS,1723024230000,1723024740000,Pl dels Indians,Espronceda - Gran Via,Pg. Marítim / Canyelles,2098.52,TMB,bc893ea5-7fd2-454c-9d30-b1dde596a8e2,1512,0,"[WALK, BUS]"
3,WALK,1723024755000,1723025537000,Espronceda - Gran Via,Destination,,848.61,None,bc893ea5-7fd2-454c-9d30-b1dde596a8e2,1512,0,"[WALK, BUS]"
0,WALK,1723023780000,1723025812000,Origin,Destination,,2437.03,None,f43dd090-201b-4f7f-8fe0-c35d0f9256c0,2032,0,[WALK]
